In [ ]:
import sys
# This makes the gdc19 package avaialble
sys.path.append('../')
import gdc19

In [ ]:
import pandas as pd
import numpy as np
import vtki
import omfvtk
import PVGeo

-----

# Initial Inspection

This notebook loads up all the data and works through an initial inspection.

In [ ]:
proj = gdc19.load_project()
proj

In [ ]:
# Add values to the meshes so colors are consistent with other data
# proj['temp_175c'].point_arrays['Temperature'] = np.full(proj['temp_175c'].n_points, 175.)
# proj['temp_225c'].point_arrays['Temperature'] = np.full(proj['temp_225c'].n_points, 225.)

In [ ]:
topo = proj['land_surface']
topo

In [ ]:
# Load kriged temperature model from Lane
# Read the kirgged model and variance
temp_grid = gdc19.load_kriged_temp()

# Remove values above topography
temp_grid = PVGeo.grids.ExtractTopography(
                remove=True, # remove the inactive cells
                tolerance=10.0 # buffer around the topo surface
               ).Apply(temp_grid, topo)
temp_grid

# Integrated Scene

Now use `vtki` to create and integrated rendering of all the available data


In [ ]:
vtki.close_all()
p = vtki.BackgroundPlotter()

In [ ]:
p.show_grid()

## Topography & Imagery

In [ ]:
# We can update this cell to change the texture
p.add_mesh(topo, 
           texture='geo_aer', 
           name='topo')

## Temperature Data

In [ ]:
temp_d_params = dict(cmap='coolwarm', stitle='Temperature', 
                     clim=temp_grid.get_data_range())

In [ ]:
p.add_mesh(proj['temperature'], name='temp', 
           point_size=10, render_points_as_spheres=True, 
           **temp_d_params)

In [ ]:
# add the temperature boundaries
p.add_mesh(proj['temp_175c'], name='temp_175c', 
           style='wireframe', **temp_d_params)
p.add_mesh(proj['temp_225c'], name='temp_225c', 
           style='wireframe', **temp_d_params)

In [ ]:
temp_grid.set_active_scalar('Temperature')
thresher = vtki.Threshold(temp_grid, plotter=p, 
                          display_params=dict(cmap='coolwarm', 
                                 clim=temp_grid.get_data_range(),
                                 show_scalar_bar=True, name='thresh',
                                 opacity=0.6),
                         default_params=dict(dmin=175, dmax=255))

In [ ]:
temp_grid.set_active_scalar('Temperature_var')
slicer = vtki.OrthogonalSlicer(temp_grid, plotter=p,
                               display_params=dict(cmap='bone_r', 
                                 clim=temp_grid.get_data_range(),
                                 show_scalar_bar=False, name='slices'),
                               clean=False)

## Fault Surfaces

In [ ]:
p.add_mesh(proj['negro_mag_fault'], name='negro_mag_fault',
           show_edges=False, color='blue', opacity=0.5)
p.add_mesh(proj['opal_mound_fault'], name='opal_mound_fault',
           show_edges=False, color='orange', opacity=0.5)

## ROI Boundary

Region of interest - where we can place a well

In [ ]:
# Add the shape file stuff
p.add_mesh(proj['boundary'], name='boundary', 
           color='azure', render_lines_as_tubes=False, line_width=10)

## Granite Surface Boundary

In [ ]:
# Add the granitoid boundary
#--
# Lets run a PVGeo filter to extract the topo surface from
#    the granitoid surface
ext = PVGeo.grids.ExtractTopography(
        remove=True, # remove the inactive cells
        tolerance=10.0 # buffer around the topo surface
        ).Apply(proj['top_granitoid'], topo)
granitoid = ext
#--
# Now plot it!
p.add_mesh(granitoid, name='top_granitoid',
           show_edges=False, style='surface',
           color='grey', opacity=0.75, reset_camera=False,
          )

## Well Collars

In [ ]:
well_locs = pd.read_csv(gdc19.get_well_path('well_location_from_earth_model.csv'))
foo = PVGeo.pointsToPolyData(well_locs[['x', 'y', 'z (land surface)']].values)
# p.add_mesh(foo, name='well_locations', point_size=10, color='orange')
idxs = []
for i, pt in enumerate(foo.points):
    if vtki.is_inside_bounds(pt,proj['boundary'].bounds):
        idxs.append(i)
wells_inside = well_locs.loc[idxs]
wells_inside

## Well Trajectories

In [ ]:
# Plot the wells
WELLS = gdc19.load_well_db()
proposed = vtki.MultiBlock()
proposed['well_new1'] = WELLS.pop('well_new1')
proposed['well_new2'] = WELLS.pop('well_new2')

well_5832 = WELLS.pop('well_5832')
well_5832.set_active_scalar('RHOZ')

In [ ]:
_ = p.add_mesh(WELLS, color='gray', name='WELLS', reset_camera=False)
_ = p.add_mesh(proposed, color='red', name='proposed-wells', reset_camera=False)

In [ ]:
_ = p.add_mesh(well_5832, scalars='RHOZ', name='5832', clim=[2.,3.], cmap='viridis')

## Gravity Data

In [ ]:
field_desc = {
    "name": "the individual gravity station name.",
    "lon": "station Longitude, WGS84",
    "lat": "station latitude, WGS84",
    "easting": "UTM zone 12 easting, NAD83",
    "northing": "UTM zone 12 northing, NAD83",
    "HAE": "height above ellipsoid [meter]",
    "NGVD29": "vertical datum for geoid [meter]",
    "obs": "observed gravity",
    "errg": "gravity measurement error [mGal]",
    "iztc": "inner zone terrain correction [mGal]",
    "oztc": "outer zone terrain correction [mGal]",
    "gFA": "free air gravity",
    "gSBGA": "Bouguer horizontal slab",
    "gCBGA": "Complete Bouguer anomaly",
    "Source": "data source",
}

In [ ]:
df = pd.read_csv(gdc19.get_gravity_path('Utah_FORGE_Gravity_Data.txt'))
ref = ['easting', 'northing', 'HAE']
points = df[ref]
for name in ref:
    df.pop(name)
grav = vtki.PolyData(points.values)
grav.point_arrays.update(df.to_dict('series'))
# clip the gravity survey
grav = grav.clip_box(gdc19.get_roi_bounds(), invert=False)
# and shift the z component so its not directly in the topo surface
grav.points[:, -1] += 25 # NOTE this is not good practice

In [ ]:
grav

In [ ]:
val = "gCBGA"
p.add_mesh(grav, scalars=val, name='gravity', 
           render_points_as_spheres=True, stitle=field_desc[val])



## Screenshot

In [ ]:
# _ = p.screenshot('../integrated_view.png')
# p.remove_actor('topo')
# p.export_vtkjs('integrated_view')